# Red Nauronal

In [10]:
# las opciones posibles en el juego

options = ["piedra", "tijeras", "papel"]


In [11]:
# definimis quien es el ganador de 2 jugadores
# p1 = player 1
# p2 = player 2
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
        
    return result

In [37]:
# Testeo de la función 

print('Ganador player : ', search_winner("papel", "tijeras"))

Ganador player :  2


In [13]:

test = [
    ["piedra", "piedra", 0],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print("player1: %s player2: %s Winner: %s Validation: %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

player1: piedra player2: piedra Winner: 0 Validation: 0
player1: piedra player2: tijeras Winner: 1 Validation: 1
player1: piedra player2: papel Winner: 2 Validation: 2


In [14]:
from random import choice

# Con la funcion choice() podemos hacer random a las opciones que tenemos
def get_choice():
    return choice(options)

In [15]:

for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: papel player2: tijeras Winner: 2 
player1: tijeras player2: piedra Winner: 2 
player1: tijeras player2: tijeras Winner: 0 
player1: piedra player2: papel Winner: 2 
player1: piedra player2: tijeras Winner: 1 
player1: papel player2: tijeras Winner: 2 
player1: tijeras player2: tijeras Winner: 0 
player1: tijeras player2: piedra Winner: 2 
player1: tijeras player2: piedra Winner: 2 
player1: piedra player2: piedra Winner: 0 


In [17]:
def str_to_list(option):
    if option=="piedra":
        res = [1,0,0]
    elif option=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

# Entrada de "X"
data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"]))
#Entrada de "Y" 
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [18]:
from sklearn.neural_network import MLPClassifier

In [19]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [22]:

# Entrenamos el modelo con un partida
model = clf.fit([data_X[0]], [data_y[0]])
print(model)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=True)


In [21]:

# funcion de jugar a aprender
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        # Generamos la opcion que escogio el player 1
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [23]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: papel Player2 (modelo): [0.11954144 0.13017963 0.87644258] --> papel
Comprobamos: p1 VS p2: 0
[]
[]
Score: {'win': 0, 'loose': 1} 0.0 %


In [24]:
i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 572, 'loose': 428} 57.2 %
Iter: 2 - score: {'win': 574, 'loose': 426} 57.4 %
Iter: 3 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 4 - score: {'win': 520, 'loose': 480} 52.0 %
Iter: 5 - score: {'win': 583, 'loose': 417} 58.3 %
Iter: 6 - score: {'win': 574, 'loose': 426} 57.4 %
Iter: 7 - score: {'win': 564, 'loose': 436} 56.4 %
Iter: 8 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 9 - score: {'win': 536, 'loose': 464} 53.6 %
Iter: 10 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 11 - score: {'win': 571, 'loose': 429} 57.1 %
Iter: 12 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 13 - score: {'win': 312, 'loose': 688} 31.2 %
Iter: 14 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 15 - score: {'win': 320, 'loose': 680} 32.0 %
Iter: 16 - score: {'win': 352, 'loose': 648} 35.2 %
Iter: 17 - score: {'win': 335, 'loose': 665} 33.5 %
Iter: 18 - score: {'win': 358, 'loose': 642} 35.8 %
Iter: 19 - score: {'win': 365, 'loose': 635} 36.5 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 570, 'loose': 430} 57.0 %
Iter: 159 - score: {'win': 524, 'loose': 476} 52.4 %
Iter: 160 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 161 - score: {'win': 761, 'loose': 239} 76.1 %
Iter: 162 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 163 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 164 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 165 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 166 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 167 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 168 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 169 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 170 - score: {'win': 1000, 'loose': 0} 100.0 %


In [28]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [45]:
from bokeh.plotting import figure, show

x = range(len(historic_pct))
y = historic_pct

p1 = figure(x_axis_type="datetime", title="Porcentaje de aprendizaje en cada iteración")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Iter'
p1.yaxis.axis_label = '%'

p1.line(x, y)
show(p1)

In [76]:
yo = 'papel' 

resultado = model.predict_proba([str_to_list(yo)])

print('escogiste: ', yo)
print(' ')
print(' ', 'Piedra','   ', 'Tijeras', '   ', 'Papel')
print(resultado)


escogiste:  papel
 
  Piedra     Tijeras     Papel
[[0.02916923 0.96265228 0.0241637 ]]
